# Benchmarking `MeshArrays` performance

- Started in 2018/09 (`gaelforget`)
- Reformated in 2019/08 (`gaelforget`)
- Added to repo in 2020/01 (`gaelforget`)

**2021/02/26 using julia 1.5**

_note the speedup in `GridLoad` & `LatitudeCircles` & `demo3` benchmarks_

```
gradient          274.532 μs (231 allocations: 3.69 MiB)
smooth             24.581 ms (10845 allocations: 238.25 MiB)

GridLoad               80.133 ms (8285 allocations: 663.26 MiB)
read_bin              151.670 μs (71 allocations: 2.01 MiB)
LatitudeCircles       647.875 ms (254111 allocations: 3.17 GiB)
ThroughFlow            43.826 ms (751490 allocations: 29.59 MiB)
demo3                 688.477 ms (1005446 allocations: 3.20 GiB)
```

**2019/08/08 using julia 1.1**

```
gradient          299.661 μs (318 allocations: 3.69 MiB)
smooth             34.006 ms (15572 allocations: 238.32 MiB)

GridLoad          337.105 ms (1960 allocations: 530.35 MiB)
read_bin          191.141 μs (61 allocations: 1.61 MiB)
LatitudeCircles     2.904 s (1511608 allocations: 4.42 GiB)
ThroughFlow loop   65.804 ms (752743 allocations: 24.46 MiB)
demo3               2.581 s (2264189 allocations: 4.44 GiB)
```

In [1]:
using BenchmarkTools
using MeshArrays
p=dirname(pathof(MeshArrays))
include(joinpath(p,"../examples/Demos.jl"))

┌ Info: Precompiling MeshArrays [cb8c808f-1acf-59a3-9d2b-6e38d009f683]
└ @ Base loading.jl:1278


demo3 (generic function with 2 methods)

In [2]:
cd("/Users/gforget/mywork/data/")

## Benchmark 1

This uses `demo2 / smooth` as done in `2018/09` to test exchanges and array operations.

In [3]:
γ,Γ=GridOfOnes("CubeSphere",6,100);
(Rini,Rend,DXCsm,DYCsm)=demo2(Γ);
@btime (dFLDdx, dFLDdy)=gradient(Rini,Γ);
@btime Rend=smooth(Rini,DXCsm,DYCsm,Γ);

  274.532 μs (231 allocations: 3.69 MiB)
  24.581 ms (10845 allocations: 238.25 MiB)


**2019/08/08**
```
299.661 μs (318 allocations: 3.69 MiB)
 34.006 ms (15572 allocations: 238.32 MiB)
```

## Benchmark 2

This uses `demo3` to test indexing type operations.

In [4]:
γ=GridSpec("LatLonCap","GRID_LLC90/")
Γ=GridLoad(γ)
TrspX=γ.read(γ.path*"TrspX.bin",MeshArray(γ,Float32))
TrspY=γ.read(γ.path*"TrspY.bin",MeshArray(γ,Float32))
TauX=γ.read(γ.path*"TauX.bin",MeshArray(γ,Float32))
TauY=γ.read(γ.path*"TauY.bin",MeshArray(γ,Float32))
SSH=γ.read(γ.path*"SSH.bin",MeshArray(γ,Float32))
(UV, LC, Tr)=demo3(TrspX,TrspY,Γ);

In [5]:
@btime Γ=GridLoad(γ)
@btime γ.read(γ.path*"TrspX.bin",MeshArray(γ,Float32))
@btime LC=LatitudeCircles(-89.0:89.0,Γ)
@btime for i=1:length(LC); ThroughFlow(UV,LC[i],Γ); end
@btime (UV, LC, Tr)=demo3(TrspX,TrspY,Γ);

  80.133 ms (8285 allocations: 663.26 MiB)
  151.670 μs (71 allocations: 2.01 MiB)
  647.875 ms (254111 allocations: 3.17 GiB)
  43.826 ms (751490 allocations: 29.59 MiB)
  688.477 ms (1005446 allocations: 3.20 GiB)


**2019/08/08**
```
257.936 ms (1960 allocations: 530.35 MiB)
177.847 μs (61 allocations: 1.61 MiB)
  2.297 s  (1511608 allocations: 4.42 GiB)
 60.967 ms (752743 allocations: 24.46 MiB)
  2.305 s  (2264189 allocations: 4.44 GiB)
```